In [7]:
import numpy
import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import tqdm

# Reproduced Model B from Spatially Transformed Adversarial Examples paper by Xiao et al. (2018)

Assignment 3 from the Deep Learning course (TU Delft) used as a guidance for the code

In [8]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))])

train_data = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST('./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=1000)

In [9]:
# Create model as described in the paper
model_b = nn.Sequential(
    nn.Conv2d(1,64,8),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Conv2d(64,128,6),
    nn.ReLU(),
    nn.Conv2d(128,128,5),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Flatten(start_dim=1),
    nn.Linear(18432,10),
    nn.Softmax(dim=1)
)

In [10]:
# Parameters
learning_rate = 0.01
epochs = 10

# Initialize
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_b.parameters(), lr=learning_rate)
scheduler_lr = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# For speed
if torch.cuda.is_available():
      device = torch.device('cuda:0')
else:
      device = torch.device('cpu')

for epoch in tqdm.trange(epochs):
    t_correct, t_samples = 0, 0
    # Train mode
    model_b.train()
    model_b.to(device)

    for i, (x_batch, y_batch) in enumerate(train_loader):
        # Set to same device (for speed)
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)

        # Set the gradients to zero
        optimizer.zero_grad()

        # Perform forward pass
        y_pred = model_b(x_batch)

        # Compute the loss
        loss = loss_fn(y_pred, y_batch)

        # Backward computation and update
        loss.backward()
        optimizer.step()

        # For accuracy calculation
        _, predicted = torch.max(y_pred, dim=1)
        t_correct += (predicted == y_batch).sum().item()
        t_samples +=  y_batch.size(0)

    # Learning rate decay step
    scheduler_lr.step()

    # Calculate accuracy
    accuracy = 100 * t_correct / t_samples
    print(f'Epoch {epoch+1}/{epochs}: accuracy = {accuracy:.2f}%, loss = {loss}')

 10%|█         | 1/10 [00:24<03:44, 24.96s/it]

Epoch 1/10: accuracy = 75.88%, loss = 1.616821527481079


 20%|██        | 2/10 [00:48<03:12, 24.02s/it]

Epoch 2/10: accuracy = 91.52%, loss = 1.4978842735290527


 30%|███       | 3/10 [01:11<02:44, 23.48s/it]

...

Epoch 8/10: accuracy = 97.06%, loss = 1.4617412090301514


 90%|█████████ | 9/10 [03:28<00:23, 23.04s/it]

Epoch 9/10: accuracy = 97.15%, loss = 1.48948335647583


100%|██████████| 10/10 [03:51<00:00, 23.20s/it]

Epoch 10/10: accuracy = 97.11%, loss = 1.4654364585876465


In [11]:
# Evaluate model with the test set
model_b.eval()
t_correct_test, t_samples_test = 0, 0

with torch.no_grad():
    for x_batch_test, y_batch_test in test_loader:
        # Set to same device (for speed)
        x_batch_test, y_batch_test = x_batch_test.to(device), y_batch_test.to(device)

        y_pred_test = model_b(x_batch_test)

        # For accuracy calculation
        _, predicted_test = torch.max(y_pred_test, dim=1)
        t_correct_test += (predicted_test == y_batch_test).sum().item()
        t_samples_test +=  y_batch_test.size(0)

# Calculate accuracy
t_accuracy = 100 * t_correct_test / t_samples_test
print(f'Test accuracy: {t_accuracy:.2f}%')

Test accuracy: 97.62%


In [12]:
# Save
torch.save(model_b.state_dict(), 'model_B.pth')